In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
import keras
import tensorflow as tf

In [3]:
data_path_and_labels = pd.read_csv("../mapped_data/ant-1.7.csv")
data_path_and_labels

,data_path,bugs
0,dataset/promise/defect data/ant-1.7/apache-ant...,1
1,dataset/promise/defect data/ant-1.7/apache-ant...,0
2,dataset/promise/defect data/ant-1.7/apache-ant...,1
3,dataset/promise/defect data/ant-1.7/apache-ant...,0
4,dataset/promise/defect data/ant-1.7/apache-ant...,7
...,...,...
736,dataset/promise/defect data/ant-1.7/apache-ant...,0
737,dataset/promise/defect data/ant-1.7/apache-ant...,0
738,dataset/promise/defect data/ant-1.7/apache-ant...,0
739,dataset/promise/defect data/ant-1.7/apache-ant...,6


In [4]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels

,data_path,bugs
0,../dataset/promise/defect data/ant-1.7/apache-...,1
1,../dataset/promise/defect data/ant-1.7/apache-...,0
2,../dataset/promise/defect data/ant-1.7/apache-...,1
3,../dataset/promise/defect data/ant-1.7/apache-...,0
4,../dataset/promise/defect data/ant-1.7/apache-...,7
...,...,...
736,../dataset/promise/defect data/ant-1.7/apache-...,0
737,../dataset/promise/defect data/ant-1.7/apache-...,0
738,../dataset/promise/defect data/ant-1.7/apache-...,0
739,../dataset/promise/defect data/ant-1.7/apache-...,6


In [5]:
with open('/home/bavanya/Desktop/AST_text_regression/dataset/promise/defect data/ant-1.3/src/main/org/apache/tools/tar/TarBuffer.java.restrictedcontent.embed') as f:
    lines = f.read().splitlines()

In [6]:
embeddings = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    embeddings.append(lines)

In [7]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,3368,3369,3370,3371,3372,3373,3374,3375,3376,3377
0,2480.0,6397.0,10419.0,2480.0,10419.0,6397.0,10419.0,10419.0,2480.0,3202.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2470.0,5355.0,2480.0,5455.0,2480.0,5065.0,2480.0,5453.0,3200.0,5063.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2338.0,2480.0,6072.0,9002.0,6072.0,2480.0,6072.0,6072.0,5355.0,2480.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2333.0,6397.0,1709.0,6397.0,1348.0,6397.0,874.0,6397.0,574.0,6397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2471.0,1123.0,4331.0,1123.0,1123.0,1123.0,1672.0,6938.0,6074.0,6938.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,2087.0,231.0,8254.0,2107.0,8494.0,2107.0,2087.0,6253.0,1196.0,8254.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
737,356.0,6397.0,3665.0,356.0,6356.0,6356.0,5937.0,5318.0,6397.0,8513.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
738,197.0,6397.0,3038.0,6397.0,1426.0,6397.0,1430.0,6397.0,1866.0,6397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
739,253.0,6397.0,6397.0,1427.0,6397.0,1959.0,6397.0,276.0,1959.0,6397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,3368,3369,3370,3371,3372,3373,3374,3375,3376,3377
0,1,2480.0,6397.0,10419.0,2480.0,10419.0,6397.0,10419.0,10419.0,2480.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2470.0,5355.0,2480.0,5455.0,2480.0,5065.0,2480.0,5453.0,3200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2338.0,2480.0,6072.0,9002.0,6072.0,2480.0,6072.0,6072.0,5355.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2333.0,6397.0,1709.0,6397.0,1348.0,6397.0,874.0,6397.0,574.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,2471.0,1123.0,4331.0,1123.0,1123.0,1123.0,1672.0,6938.0,6074.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,0,2087.0,231.0,8254.0,2107.0,8494.0,2107.0,2087.0,6253.0,1196.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
737,0,356.0,6397.0,3665.0,356.0,6356.0,6356.0,5937.0,5318.0,6397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
738,0,197.0,6397.0,3038.0,6397.0,1426.0,6397.0,1430.0,6397.0,1866.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
739,6,253.0,6397.0,6397.0,1427.0,6397.0,1959.0,6397.0,276.0,1959.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
train, test = train_test_split(data, test_size=0.2)

In [10]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,3368,3369,3370,3371,3372,3373,3374,3375,3376,3377
92,0,827.0,9199.0,7707.0,1685.0,9198.0,6421.0,1196.0,9200.0,6421.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,0,2980.0,4635.0,986.0,7723.0,1785.0,6187.0,4490.0,3249.0,3098.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
713,0,627.0,2724.0,9511.0,1758.0,2586.0,3419.0,1758.0,2714.0,3417.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
443,1,2096.0,1247.0,10058.0,1255.0,7926.0,6397.0,10218.0,6397.0,1455.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
453,0,1981.0,2106.0,324.0,2106.0,314.0,2106.0,315.0,2106.0,307.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0,2084.0,4453.0,3111.0,6897.0,1479.0,426.0,3463.0,9387.0,2106.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
364,1,2419.0,2106.0,1735.0,2106.0,1736.0,2106.0,1738.0,2106.0,1739.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
677,0,2115.0,2115.0,2106.0,1346.0,5647.0,6800.0,2377.0,3873.0,2106.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141,1,133.0,3111.0,3530.0,2106.0,4588.0,1758.0,4590.0,2377.0,7686.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,3368,3369,3370,3371,3372,3373,3374,3375,3376,3377
480,0,1878.0,3111.0,10388.0,3111.0,10389.0,3111.0,10387.0,219.0,6902.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,0,986.0,3216.0,986.0,3041.0,2377.0,4712.0,2377.0,3111.0,4132.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
321,0,3617.0,2106.0,7347.0,2106.0,8513.0,8405.0,8405.0,2106.0,2106.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396,0,1481.0,2106.0,9570.0,2106.0,10474.0,2106.0,10475.0,2106.0,8755.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
262,1,2044.0,3200.0,1345.0,6397.0,174.0,2106.0,8344.0,986.0,6877.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,0,1004.0,986.0,3052.0,2106.0,4691.0,6397.0,7994.0,1004.0,1004.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
599,0,1007.0,6565.0,3111.0,3052.0,986.0,4685.0,2106.0,4635.0,986.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
405,0,843.0,142.0,4556.0,142.0,142.0,4557.0,142.0,2647.0,4551.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0,1521.0,1008.0,4708.0,1008.0,9201.0,8098.0,1794.0,9201.0,8098.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
Y_train = train[' bugs']
Y_test = test[' bugs']

In [13]:
train_y = Y_train.to_numpy()
test_y = Y_test.to_numpy()

In [14]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [15]:
X_train = train.values
X_test = test.values

In [16]:
train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [17]:
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [18]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
19/19 [==============================] - 9s 56ms/step - loss: 1.4513 - mse: 1.4513 - mae: 0.4436 - root_mean_squared_error: 1.2047 - mean_squared_logarithmic_error: 0.2552
Epoch 2/100
19/19 [==============================] - 1s 32ms/step - loss: 1.4445 - mse: 1.4445 - mae: 0.4463 - root_mean_squared_error: 1.2019 - mean_squared_logarithmic_error: 0.2519
Epoch 3/100
19/19 [==============================] - 1s 32ms/step - loss: 1.4363 - mse: 1.4363 - mae: 0.4497 - root_mean_squared_error: 1.1984 - mean_squared_logarithmic_error: 0.2481
Epoch 4/100
19/19 [==============================] - 1s 33ms/step - loss: 1.4268 - mse: 1.4268 - mae: 0.4532 - root_mean_squared_error: 1.1945 - mean_squared_logarithmic_error: 0.2437
Epoch 5/100
19/19 [==============================] - 1s 35ms/step - loss: 1.4159 - mse: 1.4159 - mae: 0.4570 - root_mean_squared_error: 1.1899 - mean_squared_logarithmic_error: 0.2388
Epoch 6/100
19/19 [==============================] - 1s 32ms/step - loss: 1.4037

19/19 [==============================] - 1s 29ms/step - loss: 0.7180 - mse: 0.7180 - mae: 0.4162 - root_mean_squared_error: 0.8473 - mean_squared_logarithmic_error: 0.1185
Epoch 45/100
19/19 [==============================] - 0s 25ms/step - loss: 0.7149 - mse: 0.7149 - mae: 0.4145 - root_mean_squared_error: 0.8455 - mean_squared_logarithmic_error: 0.1171
Epoch 46/100
19/19 [==============================] - 0s 26ms/step - loss: 0.7127 - mse: 0.7127 - mae: 0.4147 - root_mean_squared_error: 0.8442 - mean_squared_logarithmic_error: 0.1170
Epoch 47/100
19/19 [==============================] - 1s 28ms/step - loss: 0.7052 - mse: 0.7052 - mae: 0.4098 - root_mean_squared_error: 0.8398 - mean_squared_logarithmic_error: 0.1158
Epoch 48/100
19/19 [==============================] - 1s 30ms/step - loss: 0.7119 - mse: 0.7119 - mae: 0.4109 - root_mean_squared_error: 0.8437 - mean_squared_logarithmic_error: 0.1172
Epoch 49/100
19/19 [==============================] - 0s 26ms/step - loss: 0.6959 - mse:

19/19 [==============================] - 1s 33ms/step - loss: 0.4838 - mse: 0.4838 - mae: 0.3371 - root_mean_squared_error: 0.6956 - mean_squared_logarithmic_error: 0.0858
Epoch 88/100
19/19 [==============================] - 1s 31ms/step - loss: 0.4808 - mse: 0.4808 - mae: 0.3354 - root_mean_squared_error: 0.6934 - mean_squared_logarithmic_error: 0.0864
Epoch 89/100
19/19 [==============================] - 1s 32ms/step - loss: 0.4598 - mse: 0.4598 - mae: 0.3262 - root_mean_squared_error: 0.6781 - mean_squared_logarithmic_error: 0.0823
Epoch 90/100
19/19 [==============================] - 1s 31ms/step - loss: 0.4462 - mse: 0.4462 - mae: 0.3214 - root_mean_squared_error: 0.6679 - mean_squared_logarithmic_error: 0.0812
Epoch 91/100
19/19 [==============================] - 1s 32ms/step - loss: 0.4593 - mse: 0.4593 - mae: 0.3305 - root_mean_squared_error: 0.6777 - mean_squared_logarithmic_error: 0.0856
Epoch 92/100
19/19 [==============================] - 1s 32ms/step - loss: 0.4625 - mse:

In [19]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

UsageError: Line magic function `%tensorboard` not found.


In [20]:
score = model.evaluate(test_x, test_y)
score

5/5 [==============================] - 1s 9ms/step - loss: 1.3662 - mse: 1.3662 - mae: 0.6387 - root_mean_squared_error: 1.1688 - mean_squared_logarithmic_error: 0.2504


[1.366152048110962,
 1.366152048110962,
 0.638694703578949,
 1.1688250303268433,
 0.25039994716644287]